The 2nd step "adding feasibility cut" of are as same as the 2nd step in Bneders-single Cut Alogirthm. The main differences are coming from the step of "adding optimality cut".

In [1]:
using JuMP;
using Clp;

xi = [4 4;4 8;6 4;6 8]; # combine the two xi together w.s.t probability 1/4
Master = Model(solver = ClpSolver());
@variable(Master,x[1:2]>=0);
@variable(Master,theta[1:4]>=-200);
@objective(Master,Min,3*x[1]+2*x[2]+sum(theta[k]*1.0/4 for k=1:4));

########## step 2 ########## Feasibility Cut
# for some k
k = 4;
# arbirtrary xval
xval= [0 0];
Subprob_fbt = Model(solver = ClpSolver());
@variable(Subprob_fbt, y[1:2]>=0);
@variable(Subprob_fbt, v1[1:6]>=0);
@variable(Subprob_fbt, v2[1:6]>=0);
@objective(Subprob_fbt, Min, sum(v1)+sum(v2));

# RHS updating x_hat in the constraints during the iterations
@constraint(Subprob_fbt, constr1, v1[1]-v2[1]+3*y[1]+2*y[2]<=0);
@constraint(Subprob_fbt, constr2, v1[2]-v2[2]+2*y[1]+5*y[2]<=0);

@constraint(Subprob_fbt, constr3, v1[3]-v2[3]+y[1]>=0.8*xi[k,1]);
@constraint(Subprob_fbt, constr4, v1[4]-v2[4]+y[2]>=0.8*xi[k,2]);
@constraint(Subprob_fbt, constr5, v1[5]-v2[5]+y[1]<=xi[k,1]);
@constraint(Subprob_fbt, constr6, v1[6]-v2[6]+y[2]<=xi[k,2]);

w_p = 1e5
iter = 0
while w_p > 1e-5
    status = solve(Master);
    xval = getvalue(x);
    
    println("\nxval = ",xval);
    
    JuMP.setRHS(constr1, xval[1]);
    JuMP.setRHS(constr2, xval[2]);
    
    status = solve(Subprob_fbt);
    println(status)
    w_p = getobjectivevalue(Subprob_fbt);
    println("sub-problem objective value = ", w_p);
    
    duals = [getdual(constr1) getdual(constr2) getdual(constr3) getdual(constr4) getdual(constr5) getdual(constr6)];
    h = [0 0 0.8*xi[k,1] 0.8*xi[k,2] xi[k,1] xi[k,2]]';
    T = [-1 0 0 0 0 0; 0 -1 0 0 0 0]';
    D = duals*T;
    d = duals*h;
    
    println("duals = ", duals);
    println("h = ", h);
    println("D = ", D);
    println("d = ", d);
    # Feasible cut look like: D*x >= d;
    if w_p == 0
        break;
    else
        @constraint(Master, D*x .>= d);
        @printf "Feasible cut[%d]: cutcoef[1]=%f cutcoef[2]=%f cutrhs=%f" iter D[1] D[2] d[1];
    end;
    iter = iter + 1;
    println("\n================== Iteration ", iter ," Finished ==================")
end


xval = [0.0, 0.0]
Optimal
sub-problem objective value = 11.200000000000001
duals = [-0.272727 -0.0909091 1.0 1.0 0.0 0.0]
h = [0.0; 0.0; 4.8; 6.4; 6.0; 8.0]
D = [0.272727 0.0909091]
d = [11.2]
Feasible cut[0]: cutcoef[1]=0.272727 cutcoef[2]=0.090909 cutrhs=11.200000
================== Iteration 1 Finished ==================

xval = [41.0667, 0.0]
Optimal
sub-problem objective value = 11.200000000000001
duals = [0.0 -0.5 1.0 1.0 0.0 0.0]
h = [0.0; 0.0; 4.8; 6.4; 6.0; 8.0]
D = [0.0 0.5]
d = [11.2]
Feasible cut[1]: cutcoef[1]=0.000000 cutcoef[2]=0.500000 cutrhs=11.200000
================== Iteration 2 Finished ==================

xval = [33.6, 22.4]
Optimal
sub-problem objective value = 3.84
duals = [0.0 -0.2 0.4 1.0 0.0 0.0]
h = [0.0; 0.0; 4.8; 6.4; 6.0; 8.0]
D = [0.0 0.2]
d = [8.32]
Feasible cut[2]: cutcoef[1]=0.000000 cutcoef[2]=0.200000 cutrhs=8.320000
================== Iteration 3 Finished ==================

xval = [27.2, 41.6]
Optimal
sub-problem objective value = 0.0
duals = [0.

In [2]:
println(Master);
status = solve(Master);
thetaval = getvalue(theta);
println("The theta value is: ", thetaval);

Min 3 x[1] + 2 x[2] + 0.25 theta[1] + 0.25 theta[2] + 0.25 theta[3] + 0.25 theta[4]
Subject to
 0.2727272727272727 x[1] + 0.09090909090909093 x[2] >= 11.200000000000001
 0.5 x[2] >= 11.200000000000001
 0.2 x[2] >= 8.32
 x[i] >= 0 for all i in {1,2}
 theta[i] >= -200 for all i in {1,2,3,4}

The theta value is: [-200.0, -200.0, -200.0, -200.0]


In [3]:
########## step 3 ########## Optimality Cut
# need to define the sub problem for OPT Cut
Subprob_opt = Model(solver = ClpSolver());
@variable(Subprob_opt, y[1:2]>=0);
@objective(Subprob_opt, Min, -15*y[1]-12*y[2]);

@constraint(Subprob_opt, constr1, 3*y[1]+2*y[2]<=0);
@constraint(Subprob_opt, constr2, 2*y[1]+5*y[2]<=0);
@constraint(Subprob_opt, constr3, y[1]>=0);
@constraint(Subprob_opt, constr4, y[2]>=0);
@constraint(Subprob_opt, constr5, y[1]<=0);
@constraint(Subprob_opt, constr6, y[2]<=0);

iter = 0;
f_p=1e10;

while sum(thetaval)*1.0/4<f_p
    iter = iter + 1; 
    # First solve the master problem, and get x value
    status = solve(Master);
    xval = getvalue(x);
    thetaval = getvalue(theta);
    
    println("xval = ", xval);
    println("theta = ", thetaval);
    
    master_obj = getobjectivevalue(Master);

    # Just need to change the constraint, do not need to reconstruct the model
    duals = zeros(6,4);
    h     = zeros(6,4);
    
    tempf_p = 0;
    for k = 1:4
        JuMP.setRHS(constr1, xval[1]);
        JuMP.setRHS(constr2, xval[2]);
        JuMP.setRHS(constr3, 0.8*xi[k,1]);
        JuMP.setRHS(constr4, 0.8*xi[k,2]);
        JuMP.setRHS(constr5, xi[k,1]);
        JuMP.setRHS(constr6, xi[k,2]);
        
        status = solve(Subprob_opt);
        println("Sub_prob_opt Solving: ", status, " in ", k);
        tempf_p = tempf_p + getobjectivevalue(Subprob_opt)*1.0/4;
        duals[:,k] = [getdual(constr1) getdual(constr2) getdual(constr3) getdual(constr4) getdual(constr5) getdual(constr6)]';
        h[:,k] = [0 0 0.8*xi[k,1] 0.8*xi[k,2] xi[k,1] xi[k,2]];
    end
    
    if tempf_p < f_p
        f_p = tempf_p;
    end
    
    T = [-1 0 0 0 0 0; 0 -1 0 0 0 0]';
    
    println(sum(thetaval)*1.0/4>f_p);
    
    if sum(thetaval)*1.0/4>f_p
        break;
    else
        # If not converge, MUST be able to generate a cut for at least one scenario
        # Now construct cut:
        # Cut should look like: theta[k] + E[k]*x >= e[k];
        for k=1:4
            if thetaval[k]+(duals[:,k]'*T)*xval < vecdot(duals[:,k]',h[:,k])-1e-5
                iter = iter + 1;
                E = duals[:,k]'*T;
                e = vecdot(duals[:,k]',h[:,k]);
                @constraint(Master, theta[k]+E*x>=e);
                @printf "Optimal Cut[%d]: scenario = %d, cutcoef[1] = %f, cutcoef[2] = %f, cutrhs = %f\n" iter k E[1] E[2] vecdot(duals[:,k]',h[:,k]);               
            end
        end
    end
    
    println("Master Objective value = ", master_obj);
    println("\n================== Iteration ==================");
    
end

xval = [27.2, 41.6]
theta = [-200.0, -200.0, -200.0, -200.0]
Sub_prob_opt Solving: Optimal in 1
Sub_prob_opt Solving: Optimal in 2
Sub_prob_opt Solving: Optimal in 3
Sub_prob_opt Solving: Optimal in 4
false
Optimal Cut[2]: scenario = 1, cutcoef[1] = 0.000000, cutcoef[2] = 0.000000, cutrhs = -108.000000
Optimal Cut[3]: scenario = 2, cutcoef[1] = 0.000000, cutcoef[2] = 2.400000, cutrhs = -40.800000
Optimal Cut[4]: scenario = 3, cutcoef[1] = 0.000000, cutcoef[2] = 0.000000, cutrhs = -138.000000
Optimal Cut[5]: scenario = 4, cutcoef[1] = 6.000000, cutcoef[2] = 0.000000, cutrhs = 14.400000
Master Objective value = -35.19999999999999

================== Iteration ==================
xval = [27.2, 41.6]
theta = [-108.0, -140.64, -138.0, -148.8]
Sub_prob_opt Solving: Optimal in 1
Sub_prob_opt Solving: Optimal in 2
Sub_prob_opt Solving: Optimal in 3
Sub_prob_opt Solving: Optimal in 4
false
Master Objective value = 30.940000000000012

================== Iteration ==================


In [4]:
println("f(xhat) = ", f_p); # UB must be bigger than lower bound
println("theta = ", thetaval);
println("GAP = ", f_p-thetaval);
println("The optimal xval = ", xval);
println("\n==============================\n");
println(Master);

f(xhat) = -133.86
theta = [-108.0, -140.64, -138.0, -148.8]
GAP = [-25.86, 6.78, 4.14, 14.94]
The optimal xval = [27.2, 41.6]


Min 3 x[1] + 2 x[2] + 0.25 theta[1] + 0.25 theta[2] + 0.25 theta[3] + 0.25 theta[4]
Subject to
 0.2727272727272727 x[1] + 0.09090909090909093 x[2] >= 11.200000000000001
 0.5 x[2] >= 11.200000000000001
 0.2 x[2] >= 8.32
 theta[1] >= -108
 theta[2] + 2.4000000000000004 x[2] >= -40.8
 theta[3] >= -138
 theta[4] + 6 x[1] >= 14.400000000000002
 x[i] >= 0 for all i in {1,2}
 theta[i] >= -200 for all i in {1,2,3,4}

